In [236]:
import geopandas as gpd
import rasterio as rio
import rasterstats

from netCDF4 import Dataset

import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns

from mpl_toolkits.basemap import Basemap

In [66]:
#Load the Districts Shapefile
districts = gpd.read_file('Census_2011/2011_Dist.shp')

In [156]:
file_path = "RainfallData_25/2019.nc"
fh = Dataset(file_path, mode='r')
#Longitudes
lons = fh.variables['longitude'][:]
#Latitudes
lats = fh.variables['latitude'][:]
#Time Series
times = fh.variables['time'][:]
#Total Precipitation Data 
tps = fh.variables['tp'][:]


affine = rio.open(file_path).transform

In [157]:
#Total Precipitation for January 2019
tp_jan = tps[0,:,:]
tp_jan

masked_array(
  data=[[0.00020628, 0.00020628, 0.00020628, ..., 0.00020628, 0.00020628,
         0.00020628],
        [0.00027153, 0.00027153, 0.00027153, ..., 0.00027153, 0.00027153,
         0.00027153],
        [0.00027153, 0.00027153, 0.00027153, ..., 0.00027153, 0.00027153,
         0.00027153],
        ...,
        [0.00017155, 0.00017155, 0.00017155, ..., 0.00017155, 0.00017155,
         0.00017155],
        [0.00017155, 0.00017155, 0.00017155, ..., 0.00017155, 0.00017155,
         0.00017155],
        [0.00013577, 0.00013577, 0.00013577, ..., 0.00013577, 0.00013577,
         0.00013577]],
  mask=False,
  fill_value=1e+20)

All the Total Precipitation Arrays have mask = False. 

But there can still be anomalies in the data due to bad sensor. The documentation of ERA-interim read that missing values were filled with negative values. Let's check that. 

In [158]:
tp_jan.min()

-4.295636069073794e-11

In [159]:
np.count_nonzero(tp_jan == tp_jan.min())

15195

In [160]:
tp_jan.shape[0]*tp_jan.shape[1]

1038240

In [162]:
100*15195/1038240

1.463534442903375

Around 1.5% of the pixels have an anamolous value. But when we have to find zonal statistics, these anamolous values will hurt the zonal stat. Thus, we should impute these pixels. 
<br>
One approach is to impute these pixels with mean of surrounding pixel values

## Imputing Missing pixel values - Explainer. 

In [163]:
from astropy.convolution import convolve

In [164]:
Y =[[1,5,2,9,3,2],
    [6,np.nan,7,3,9,3],
    [7,2,3,9,8,2],
    [2,1,4,np.nan,5,2],
    [5,2,8,9,2,4]]
Y = np.array(Y)
np.squeeze(Y)

array([[ 1.,  5.,  2.,  9.,  3.,  2.],
       [ 6., nan,  7.,  3.,  9.,  3.],
       [ 7.,  2.,  3.,  9.,  8.,  2.],
       [ 2.,  1.,  4., nan,  5.,  2.],
       [ 5.,  2.,  8.,  9.,  2.,  4.]])

In [165]:
kernel = [[1,1,1],
          [1,0,1],
          [1,1,1]]

In [166]:
#Mask Null Values
mask = np.isnan(Y)

In [167]:
#Mean of Neigbouring elements: Won't work effectively for edges. 
##So choose satellite image that has a little bigger boundary than the focus area.
means = convolve(Y,kernel)
np.squeeze(means)

array([[1.57142857, 2.28571429, 3.42857143, 3.        , 3.25      ,
        1.875     ],
       [2.14285714, 4.125     , 4.71428571, 6.25      , 4.875     ,
        3.        ],
       [1.57142857, 4.28571429, 4.33333333, 5.57142857, 4.71428571,
        3.375     ],
       [2.125     , 4.125     , 4.85714286, 6.        , 5.14285714,
        2.625     ],
       [0.625     , 2.5       , 2.28571429, 2.71428571, 2.85714286,
        1.125     ]])

In [168]:
mask_inv = np.bitwise_not(mask)
mask_inv

array([[ True,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True, False,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [169]:
means[mask_inv] = np.zeros((mask_inv.sum()))
means

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 4.125, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 6.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]])

In [170]:
Y[mask] = np.zeros((np.array(mask).sum()))
Y

array([[1., 5., 2., 9., 3., 2.],
       [6., 0., 7., 3., 9., 3.],
       [7., 2., 3., 9., 8., 2.],
       [2., 1., 4., 0., 5., 2.],
       [5., 2., 8., 9., 2., 4.]])

In [171]:
Y_imputed = Y +means
Y_imputed

array([[1.   , 5.   , 2.   , 9.   , 3.   , 2.   ],
       [6.   , 4.125, 7.   , 3.   , 9.   , 3.   ],
       [7.   , 2.   , 3.   , 9.   , 8.   , 2.   ],
       [2.   , 1.   , 4.   , 6.   , 5.   , 2.   ],
       [5.   , 2.   , 8.   , 9.   , 2.   , 4.   ]])

In [172]:
#Let's write a function for this:
def impute_anomalies(Y,smoothing_iter = 1):
    Y = np.array(Y)
    #Replace anomalous values with np.nan
    Y = np.where(Y<0, np.nan, Y)
    
    #Bigger kernel only to deal with successive nulls
    kernel = [[0.0001,0.0001,0.0001,0.0001,0.0001],
              [0.0001,1,1,1,0.0001],
              [0.0001,1,0,1,0.0001],
              [0.0001,1,1,1,0.0001],
              [0.0001,0.0001,0.0001,0.0001,0.0001]]
    
    for i in range(smoothing_iter):
        #Mask Null Values
        mask = np.isnan(Y)
        
        means = convolve(Y,kernel)
        
        mask_inv = np.bitwise_not(mask)
        
        means[mask_inv] = np.zeros((mask_inv.sum()))
        
        Y[mask] = np.zeros((np.array(mask).sum()))
        Y_imputed = Y +means
        Y = Y_imputed
    
    return Y_imputed

In [173]:
# Let's check the function with tp_jan
for i in range(5):
    tp_jan = impute_anomalies(tp_jan,smoothing_iter=i+1)
    print('After iteration', i+1)
    print('Number of Missing Values:',np.isnan(tp_jan).sum())
    

After iteration 1
Number of Missing Values: 10693


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """


After iteration 2
Number of Missing Values: 5607
After iteration 3
Number of Missing Values: 2134
After iteration 4
Number of Missing Values: 444
After iteration 5
Number of Missing Values: 0


Hence 5 iterations are needed to fill all anomalous pixel values in the data. 

# Calculate Zonal Statistics:

We will calculate mean rainfall in each district of India. 
<br>
We will also count number of pixels that fall under each district. 
<br>

Rasterstats will help us do it. 

In [179]:
df = pd.DataFrame(columns=['Census Code','State Code','District Code',"State","District","Month","Year","Mean Rainfall","Count"])
for t in range(8):
    print(t)
    precipitation = tps[t,:,:]
    precipitation = impute_anomalies(precipitation,smoothing_iter=5)
    
    average_tp = rasterstats.zonal_stats(districts,
                                             precipitation,
                                             affine=affine,
                                             stats = ['mean','count'],
                                             geojson_out=True)
    for j in range(len(average_tp)):
                df = df.append({'Census Code':average_tp[j]['properties']['censuscode'],
                            'State Code':average_tp[j]['properties']['ST_CEN_CD'],
                            'District Code':average_tp[j]['properties']['DT_CEN_CD'],
                            'District':average_tp[j]['properties']['DISTRICT'],
                            'State':average_tp[j]['properties']['ST_NM'],
                            'Month':t+1,
                            'Year':2019,
                            'Mean Rainfall':average_tp[j]['properties']['mean'],
                            'Count':average_tp[j]['properties']['count']}, ignore_index=True)

0
1
2
3
4
5
6
7


In [180]:
df.sample(5)

,Census Code,State Code,District Code,State,District,Month,Year,Mean Rainfall,Count
1108,408,22,9,Chhattisgarh,Rajnandgaon,2,2019,0.000285595,11
4207,281,15,1,Mizoram,Mamit,7,2019,0.00943304,3
4152,576,29,22,Karnataka,Kodagu,7,2019,0.0168207,4
1325,170,9,39,Uttar Pradesh,Banda,3,2019,0.000527011,8
2361,230,10,28,Bihar,Patna,4,2019,0.000993506,3


In [192]:
df['Day'] = 1
df['Date'] = pd.to_datetime(df[['Year','Month','Day']])

In [201]:
df['days_in_month'] = df['Date'].dt.daysinmonth
df.sample(4)

,Date,Census Code,State Code,District Code,State,District,Month,Year,Mean Rainfall,Count,Day,days_in_month
2103,2019-04-01,159,9,28,Uttar Pradesh,Farrukhabad,4,2019,0.000355726,2,1,30
1660,2019-03-01,135,9,4,Uttar Pradesh,Moradabad,3,2019,0.000280651,6,1,31
2463,2019-04-01,76,6,8,Haryana,Sonipat,4,2019,0.000656373,3,1,30
920,2019-02-01,164,9,33,Uttar Pradesh,Kanpur Nagar,2,2019,0.00200885,4,1,28


In [221]:
df['Mean Rainfall'] =  df['Mean Rainfall']*1000
df['Total Rainfall'] = df['Mean Rainfall']*df['days_in_month']

In [222]:
df_geom = pd.merge(df,districts[['censuscode','geometry']],left_on = 'Census Code', right_on='censuscode').drop('censuscode',axis=1)
df_geom.sample(4)

,Date,Census Code,State Code,District Code,State,District,Month,Year,Mean Rainfall,Count,Day,days_in_month,Total Rainfall,geometry
1792,2019-01-01,564,29,10,Karnataka,Haveri,1,2019,0.11219,5,1,31,3.4779,"POLYGON ((75.41951 15.06168, 75.42000 15.06289..."
2924,2019-05-01,573,29,19,Karnataka,Mandya,5,2019,5.40798,8,1,31,167.647,"MULTIPOLYGON (((76.68466 13.04283, 76.69453 13..."
3393,2019-02-01,118,8,20,Rajasthan,Pali,2,2019,0.0569557,17,1,28,1.59476,"POLYGON ((74.29076 26.44896, 74.28900 26.44316..."
3931,2019-04-01,352,20,7,Jharkhand,Sahibganj,4,2019,3.20469,4,1,30,96.1406,"POLYGON ((87.71357 25.25694, 87.72683 25.25397..."


In [223]:
df_geom = df_geom.set_index('Date')
df_geom.head(3)#.plot(column=)

,Census Code,State Code,District Code,State,District,Month,Year,Mean Rainfall,Count,Day,days_in_month,Total Rainfall,geometry
Date,,,,,,,,,,,,,
2019-01-01,532,28,1,Andhra Pradesh,Adilabad,1,2019,1.31446,23,1,31,40.7481,"POLYGON ((78.84972 19.76010, 78.85102 19.75945..."
2019-02-01,532,28,1,Andhra Pradesh,Adilabad,2,2019,0.153885,23,1,28,4.30879,"POLYGON ((78.84972 19.76010, 78.85102 19.75945..."
2019-03-01,532,28,1,Andhra Pradesh,Adilabad,3,2019,0.297475,23,1,31,9.22173,"POLYGON ((78.84972 19.76010, 78.85102 19.75945..."


In [231]:
gdf = gpd.GeoDataFrame(df_geom)
convert_dict = {'Mean Rainfall': float, 
                'Total Rainfall': float
               } 
  
gdf = gdf.astype(convert_dict) 

In [238]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

In [241]:
midnorm = MidpointNormalize(vmin=0, vcenter=100, vmax=700)

In [250]:
months=['January','February','March','April','May','June','July','August']
for i in range(len(months)):
    print(months[i])
    date = '2019-0'+str(i+1)
    gdf[date].plot(column='Total Rainfall', cmap='RdBu', figsize=(10,10),legend=True, norm=midnorm,clim=(0,700))
    title = 'Rainfall in (mm): '+months[i]
    plt.title(title,fontsize=20)
    plt.savefig(date+'.png', bbox_inches='tight')
    plt.close('all')

January
February
March
April
May
June
July
August
